In [1]:
import argparse
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import math
from IPython.display import display
from multiprocessing import cpu_count,Pool 
from joblib import Parallel, delayed
import psutil
import multiprocessing
import time

In [9]:
print("Load long events")
long_events_ = pd.read_hdf('../long_events_.h5',key='LN_300_nc2')
print ("*"*80)
print (long_events_.shape)
long_events_.head()

Load long events
********************************************************************************
(148237, 18)


,EventId,Type,RefinedType_,StartTime,EndTime,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode,RefinedType,duration
6538970,T-5006887,T,Construction,2018-07-12 09:52:41,2018-07-31 10:22:00,34.189785,-86.526672,0.000000,K8A0,1267.0,State Highway 67,L,Blountsville,Blount,AL,35031.0,Construction,27389.316667
27262,W-36847,W,cold-severe,2018-05-24 16:56:00,2018-06-12 14:56:00,NaN,NaN,NaN,K1M4,NaN,NaN,NaN,Russellville,NaN,AL,NaN,cold,27240.000000
6772201,T-5918907_T-6202981_T-6203021,T,Construction,2018-07-12 09:37:25,2018-07-31 05:00:00,32.600131,-85.432367,67.610001,KAUO,NaN,Columbus Pkwy,R,Opelika,Lee,AL,36804.0,Congestion_Construction,27082.583333
7917957,T-7073427_W-74530_W-74545,T_W,Construction,2018-04-19 14:35:45,2018-05-07 04:59:00,34.824619,-85.785294,4.360000,K4A6,8271.0,AL Highway 117,L,Flat Rock,Jackson,AL,35966.0,Construction_rain,25343.250000
8636738,T-7797145_T-7797146,T,Construction,2018-03-28 15:44:59,2018-04-15 04:59:00,31.542608,-86.220195,6.660000,KLOR,4498.0,Elba Hwy,L,Elba,Crenshaw,AL,36323.0,Construction,25274.016667


In [3]:
import pickle
with open("test.txt", "rb") as fp:
    final_data_frame = pickle.load(fp)

In [8]:

file = "AirportWithNoTrafficDataBefore25July2017.txt"
airport_lists = pd.read_csv(file,low_memory=True)
#airport_lists = np.loadtxt(file)
airport_lists.head()

,Airports
0,K9MN
1,KDDC
2,KVQQ
3,KBXA
4,KOTG


In [30]:
mask_arr=[]
time = pd.to_datetime("2017-07-01 00:00:00", format="%Y-%m-%d %H:%M:%S", errors='coerce')
print (time)
counter=0
for idx,long_event in long_events_.iterrows():
    #print (idx)
    if long_event.Type == 'W' and (long_event.AirportCode in airport_lists.Airports.values )and long_event.EndTime < time:
        counter+=1
        mask_arr.append(1)
    else:
        mask_arr.append(0)
print (counter)
print (len(mask_arr))

2017-07-01 00:00:00
29771
148237


In [19]:
long_events_[long_events_.EndTime<"2018-07-12 09:52:41"]

,EventId,Type,RefinedType_,StartTime,EndTime,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode,RefinedType,duration
27262,W-36847,W,cold-severe,2018-05-24 16:56:00,2018-06-12 14:56:00,NaN,NaN,NaN,K1M4,NaN,NaN,NaN,Russellville,NaN,AL,NaN,cold,27240.000000
7917957,T-7073427_W-74530_W-74545,T_W,Construction,2018-04-19 14:35:45,2018-05-07 04:59:00,34.824619,-85.785294,4.360000,K4A6,8271.0,AL Highway 117,L,Flat Rock,Jackson,AL,35966.0,Construction_rain,25343.250000
8636738,T-7797145_T-7797146,T,Construction,2018-03-28 15:44:59,2018-04-15 04:59:00,31.542608,-86.220195,6.660000,KLOR,4498.0,Elba Hwy,L,Elba,Crenshaw,AL,36323.0,Construction,25274.016667
13716854,T-12913075_T-14134756_T-14233188_T-14325334,T,Construction,2017-08-15 14:02:28,2017-09-01 04:59:00,34.662990,-86.803953,0.000000,KHUA,857.0,Mill Rd,L,Madison,Madison,AL,35758.0,Construction_Other,23936.533333
7192550,T-6342603,T,Construction,2018-05-15 14:16:36,2018-06-01 03:59:00,32.502323,-85.048828,0.000000,KCSG,NaN,Price Rd,R,Phenix City,Russell,AL,36870.0,Construction,23862.400000
7198957,T-6349038_T-7438981_T-7438982_T-7439443_T-7439...,T_W,Construction,2018-05-15 15:26:59,2018-06-01 05:00:00,31.867370,-86.620505,4.100000,KPRN,NaN,Fort Dale Rd,R,Greenville,Butler,AL,36037.0,Construction_rain,23853.016667
7199476,T-6349561_T-7439445_T-7439446_W-1854911,T_W,Construction,2018-05-15 15:19:28,2018-05-31 23:00:00,32.166855,-86.432257,9.870000,KMGM,NaN,I-65 S,R,Hope Hull,Lowndes,AL,36043.0,Construction_fog,23500.533333
8635873,T-7796274_T-7796275_T-7796276,T,Construction,2018-03-16 09:21:28,2018-04-01 04:59:00,30.852203,-88.064349,4.040000,KBFM,NaN,I-65 N,R,Saraland,Mobile,AL,36571.0,Construction,22777.533333
8281104,T-6349042_T-7438953_T-7438954_T-7438956_T-7438...,T,Construction,2018-05-14 14:19:00,2018-05-31 16:20:00,33.507861,-86.800907,0.000000,KBHM,898.0,12th Ct W,R,Birmingham,Jefferson,AL,35204.0,Congestion_Construction_Event_Incident,24601.000000
9736622,T-8904408_T-8904409_T-8904414_T-8904415_T-8904...,T,Construction,2018-03-15 14:00:00,2018-03-31 04:59:00,33.453090,-86.839517,0.000000,KBHM,1101.0,8th St W,R,Birmingham,Jefferson,AL,35204.0,Construction,22499.000000


In [26]:
long_events_.loc[27262]

EventId                     W-36847
Type                              W
RefinedType_            cold-severe
StartTime       2018-05-24 16:56:00
EndTime         2018-06-12 14:56:00
LocationLat                     NaN
LocationLng                     NaN
Distance(mi)                    NaN
AirportCode                    K1M4
Number                          NaN
Street                          NaN
Side                            NaN
City                   Russellville
County                          NaN
State                            AL
ZipCode                         NaN
RefinedType                    cold
duration                      27240
Name: 27262, dtype: object